In [36]:
import os
import numpy as np
from matplotlib import pyplot as plt
import nmrglue as ng
import scipy as sp
import scipy.optimize as opt

In [29]:
def org_T1(numTR,path,folder,t1):  
    
    '''
    This function organizes raw spectroscopially acquired t1-vtr data, into a list comprising a nd-array
    containing the signal intensities corresponding to each TR for every scan in that experiment. To be used
    for T1 calculation
    
    Args:
        numTR (int): number of repetitions in T1-acquisition
        path (str): path to folder containing data under different conditions
        folder (list): containing directory names (str) of Bruker raw data under different conditions
        res (int) : Number of TR images that are acquired before the null point
        t1 (float) : Estimated T1 value (used for estimating how many images are acquired before crossing null point)
     '''
    
    t1Data=[]
    thresh=0.69*t1
    res=[i for i, val in enumerate(TR) if val <=thresh]

    for i in range(len(folder)):
        dirs=[d for d in os.listdir((os.path.join(path,folder[i])))]
        amps=np.zeros((len(dirs),numTR))

        for j in range(len(dirs)):
            dic,data=ng.bruker.read(os.path.join(path,folder[i],dirs[j]))

            #Data acquired as real and imaginary pairs
            count=0
            amp=np.zeros(int(data.shape[0]/2), dtype=np.complex)
            for k in range(int(data.shape[0]/2)):
                amp[k]=complex(data[count], data[count+1])
                count=count+2
            ampMag=abs(amp)
            sep=int(ampMag.shape[0]/numTR)
            peak=np.zeros((numTR,sep))
            count=0
            sepCount=sep

            for k in range(numTR):
                peak[k,:]=ampMag[count:count+sep]
                count=count+sep
                
            for k in range(numTR):
                amps[j,k]=np.max(peak[k,:])
            
            #null crossing point at 0.69*T1
            res=[k for k, val in enumerate(TR) if val <=thresh]
            lim=res[-1]+1
            amps[j,0:lim]=amps[j,0:lim]*-1
            amps[j,:]=amps[j,:]+abs(amps[j,:].min())

        t1Data.append(amps)
    return t1Data

def t1r_vtr(TR, Mo,Minf, t1):
    '''
    3-parameter inverision-recovery equation
    '''
    return Mo+(Minf-Mo)*(1-(2*(np.exp(-TR/t1))))

def T1_Spect(t1Data,TR):
    '''
    This function calculates the 1/T1 value of spectroscopically acquired t1-vtr data.
    
    Args:
        t1Data (list): list comprising an nd-array contaning the signal intensities acquired at each TR for every
        scan in that experiment (Output of org_T1).
        TR (array): containing repetition times that spectroscopic t1-vtr data were acquired at.
    '''
    t1Result=[]
    for i in range(len(folder)):
        coeffs=np.zeros((t1Data[i].shape[0],3))

        for j in range(t1Data[i].shape[0]):
            b=[np.max(t1Data[i][j,:]),np.min(t1Data[i][j,:]),2.6]
            coeffs[j,:],vmat= opt.curve_fit(t1r_vtr,TR,t1Data[i][j,:],b,method='lm')
            
        t1Result.append(1./coeffs[:,2])
    return t1Result

def calcROSConc(a0,t,pH):
    
    '''
    This function calculates the pH and time-dependent concentration (M) of the superoxide free radical 
    in aqueous solution at a certain time, t, post-dissolution. Equation for calculating reaction rate
    constant from Eq. (1) in: Kwon BG, Kim J, Kwon J. An Advanced Kinetic Method for HO2∙/O2-∙ Determination by Using 
    Terephthalate in the Aqueous Solution. Environmental Engineering Research 2012;17(4):205-210.
    Args:
        a0 (int): initial concentration (M) of superoxide in solution at t=0.
        t (int):  time at which superoxide concentration is to be calculated (s)
        pH (float): pH of solution containing superoxide (negative logarithm of hydrogen ion concentration)
    '''

    k3=8.3E5
    k4=9.76E7
    kh=1.6E-5
    hp=10**-pH
    ko=(k3+(k4*(kh/hp)))/(1+(kh/hp))**2
    at=(ko*t+(1/a0))**-1

    return at



In [30]:
#Figure 3A
'''
This script produces the data for the plot and calculate the statistics for the data as shown in Figure 3A. 
It was used to assess the impact of pH on 1/T1. See Methods section and figure legend for more detail.
'''

#Analysis setup
path=os.path.join(os.getcwd(),'data\\pH')
numTR=8
t1=2.6
TR=np.array([0.1, 0.8, 1.5, 2, 2.5, 4, 8.5, 11])
folder = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]

#Organizing data
t1Data=org_T1(numTR,path,folder,t1)

#Calculating 1/T1
t1pH=T1_Spect(t1Data,TR)

#One-way ANOVA

print(sp.stats.f_oneway(t1pH[0],t1pH[1],t1pH[2],t1pH[3]))

F_onewayResult(statistic=1.8780628619332693, pvalue=0.16582588320225702)


In [31]:
#Figure 3B
'''
This script produces the data for the plot and calculate the statistics for the data as shown in Figure 3B. 
It was used to assess the impact KO2 solution on 1/T1 and the antioxidant on KO2 solution and water 1/T1.
See Methods section and figure legend for more detail.
'''

#Analysis setup

path=os.path.join(os.getcwd(),'data\\relaxivity')
numTR=8
t1=2.6
TR=np.array([0.1, 0.8, 1.5, 2, 2.5, 4, 8.5, 11])
folder = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]

#Organizing data
t1Data=org_T1(numTR,path,folder,t1)

#Calculating 1/T1
t1KO2=T1_Spect(t1Data,TR)

#One-way ANOVA of Solutions of 1/T1 KO2 in absence of the antioxidat SOD
print(sp.stats.f_oneway(t1KO2[0],t1KO2[1],t1KO2[2],t1KO2[3],t1KO2[4],t1KO2[5]))

#One-way ANOVA of Solutions of 1/T1 KO2 in presence of the antioxidat SOD
print(sp.stats.f_oneway(t1KO2[6],t1KO2[7],t1KO2[8],t1KO2[9],t1KO2[10],t1KO2[11]))


F_onewayResult(statistic=50.520481312167426, pvalue=1.0313692823538624e-13)
F_onewayResult(statistic=0.4229161534356605, pvalue=0.8290140330694207)


In [32]:
#Figure 4
'''
This script produces the data for the plot and calculate the statistics for the data as shown in Figure 4. 
It calculates the r1 from the data in figure 3B. The 1/T1 data of KO2 in the presence of the antioxidant SOD is subtracted
from KO2 in the absence of the antioxidant SOD. The r1 is calculated from a straight line-curve fit of this subtraction.
See Methods section and figure legend for more detail.
'''

#Subtracting 1/T1 data of SOD- and SOD+ KO2 data, to calculate r1 from subtraction.
t1Sub=np.zeros((int(((len(folder))/2)),len(t1Sub)))
for i in range(int((len(t1KO2))/2)):
    t1Sub[i,:]=t1KO2[i]-t1KO2[i+6]

t1SubMean=np.mean(t1Sub,1)

'''
Setting up and carrying out calculation of time and pH dependent
superoxide in KO2 solutions at mid-point of 2 minute acquisition 
following 10 minutesample setup
'''
a0=np.linspace(4,20,5)
#pH of 4,8,12,16 and 20 mM KO2 solutions respectively (see Table 1)
pH=[11.61,11.91,12.04,12.15,12.43]
t=660
at=np.zeros(len(a0)+1)
for i in range(len(a0)):
    at[i+1]=calcROSConc(a0[i],t,pH[i])*1000
    


r1, c, r, p, stdErr = stats.linregress(at, t1SubMean)
adjrSq=1-(((1-r**2)*(36-1))/(36-6-1))

print("relaxivity is {}".format(r1))
print("interccept is {}".format(c))
print("r^2 is {}".format(r**2))
print("Adjusted r^2 is {}".format(adjrSq))
print("Standard Error is {}".format(stdErr))
print("p-value is {}".format(p))


relaxivity is 0.2897063369312174
interccept is -0.01438674313201152
r^2 is 0.952663532168576
Adjusted r^2 is 0.9428697802034538
Standard Error is 0.03228911584076144
p-value is 0.0008539011152287562


In [33]:
#Supporting Figure S3 (SOD-)

'''
This script produces the data for the plot and calculates the statistics for the data in Supporting Figure S3. 
Specifically, it plots the 1/T1 of the KO2 data in the absence of the antioxidant SOD and assesses whether there is a 
linear relationship between 1/T1 and KO2 cocentration in the SOD- group.

'''

t1KO2Mean=np.zeros((int(((len(folder))/2)),(int(((len(t1KO2))/2)))))
for i in range(int((len(t1KO2))/2)):
    t1KO2Mean[i,:]=t1KO2[i]
    
t1KO2Mean=np.mean(t1KO2Mean,1)

r1, c, r, p, stdErr = stats.linregress(at, t1KO2Mean)
adjrSq=1-(((1-r**2)*(36-1))/(36-6-1))


print("relaxivity is {}".format(r1))
print("interccept is {}".format(c))
print("r^2 is {}".format(r**2))
print("Standard Error is {}".format(stdErr))
print("Adjusted r^2 is {}".format(adjrSq))
print("p-value is {}".format(p))


relaxivity is 0.29634251321290717
interccept is 0.35133374391281047
r^2 is 0.9447393007535201
Standard Error is 0.035835722174214095
Adjusted r^2 is 0.9333060526335587
p-value is 0.0011669292020169773


In [34]:
#Supporting Figure S3 (SOD+)
'''
This script produces the data for the plot and calculates the statistics for the data as shown in 
Supporting Figure S3.It assesses whether there is a linear relationship between 1/T1 and KO2 concentration 
in the SOD+ group.

'''

t1KO2SOD=np.zeros((int(((len(folder))/2)),len(t1KO2SOD)))


for i in range(int((len(t1KO2))/2)):
    t1KO2SOD[i,:]=t1KO2[i+6]
    
t1KO2SOD=np.mean(t1KO2SOD,1)

r1, c, r, p, stdErr = stats.linregress(at, t1KO2SOD)
adjrSq=1-(((1-r**2)*(36-1))/(36-6-1))


print("relaxivity is {}".format(r1))
print("interccept is {}".format(c))
print("r^2 is {}".format(r**2))
print("Adjusted r^2 is {}".format(adjrSq))
print("Standard Error is {}".format(stdErr))
print("p-value is {}".format(p))

relaxivity is 0.006636176281689689
interccept is 0.36572048704482213
r^2 is 0.36813674225197696
Adjusted r^2 is 0.23740641306273091
Standard Error is 0.004347052518109875
p-value is 0.2015680490038245
